# Catalog Output

In [4]:
import apogee.tools.read as apread
import apogee.spec.plot as splot
import numpy as np
Lambda = splot.apStarWavegrid()
import csv
import sys

### Calculations and Definitions

In [ ]:
def find_nearest(array,value):
    index = (np.abs(array-value)).argmin()
    #return array[index]
    return index

In [ ]:
def Equiv_Width(loc_id,twomass_id):
    
    
    #given a certain spec
    spec_header = apread.apStar(loc_id,twomass_id,ext=0,header=True)
    spec_noheader = apread.apStar(loc_id,twomass_id,ext=1,header=False)
    nvisits = spec_header[1]['NVISITS']
    EqW_array = []
    
    #calculate the emission line in a vacuum
    vhelio = spec_header[1]['VHELIO']

    c = 299792
    rydberg = 1.0973731568539*(10**7)
    electron = 9.10938356*(10**-31)
    nucleus = 1.672621898*(10**-27)
    fracryd = rydberg/(1+(electron/nucleus))
    vacuum = fracryd*((1./16.)-(1./121.))
    lambda_obs = 1/vacuum
    calculated_point1 = lambda_obs*(1+(-vhelio/c))
    calculated_point2 = calculated_point1*(10**10)
    
    
    
    #EqW Calculations
    for i in range(nvisits):
        spec = spec_noheader[2 + i]
    
        centerline = find_nearest(Lambda,calculated_point2)
        L1 = centerline - 90
        L2 = centerline - 45
        R1 = centerline + 45
        R2 = centerline + 90
        #generic continuum lines 35 elements wide 
        lsum= np.sum(spec[L1:L2])/ len(spec[L1:L2])
        rsum = np.sum(spec[R1:R2])/len(spec[R1:R2])
        Fc= (lsum+rsum)/2
    
        EqW=0
        
        for i in range(L2,R1):
            summ=(Fc*(Lambda[i+1]-Lambda[i]))-((1./2.)*(Lambda[i+1]-Lambda[i])*(spec[i+1]+spec[i]))
            EqW = EqW + summ
        EqW = abs(EqW/Fc)
        EqW_array.append(EqW)
        
    #Error Calculations
    Single_StD = [] 
    avg = np.mean(EqW_array)

    #Calculating the standard dev for each element
    for i in EqW_array:
        squared = (i - avg)**2
        final=np.sqrt(squared/len(x))
        Single_StD.append(final) 
    
    #Average standard dev
    StD_avg = np.std(EqW_array)
    EqW_avg = np.median(EqW_array)
    
    return StD_avg,EqW_avg,EqW_array,Single_StD,nvisits

### Output

In [ ]:
def All_Outputs(csvname,allsavefile,avgsavefile):
    
    #INPUT
    with open(csvname) as csvfile:
        
        reader = csv.DictReader(csvfile,delimiter='\t')
    
        for row in reader:
            loc_id=row['Location ID']
            twomass_id=row['2Mass ID']
            Equiv_Width(loc_id,twomass_id)
            
            for i in range(len(nvisits)):
                all_list.append(loc_id,twomass_id,i+1,EqW_array[i],Single_StD[i])
            avg_list.append(loc_id,twomass_id,nvisits,EqW_avg,StD_avg)
    
    csvfile.close() #Maybe check this
    
    #Output for every visit
    with open(allsavefile,'w') as savefile:
        fieldnames = ['Location ID','2Mass ID','Visit #','Equivalent Width','Standard Deviation']
        writer = csv.DictWriter(savefile,delimiter='\t',fieldnames=fieldnames)
        
        writer.writeheader()
        for i in range(len(all_list)):
            writer.writerow({'Location ID': all_list[i][0],
                             '2Mass ID': all_list[i][1],
                             'Visit #': all_list[i][2],
                             'Equivalent Width': all_list[i][3],
                             'Standard Deviation': all_list[i][4]})
    savefile.close() #Maybe check this
    
    #Output for average values
    with open(avgsavefile,'w') as savefile2:
        fieldnames = ['Location ID','2Mass ID','# of Visits','Average Equivalent Width','Average Standard Deviation']
        writer = csv.DictWriter(savefile2,delimiter='\t',fieldnames=fieldnames)
        
        writer.writeheader()
        for i in range(len(avg_list)):
            writer.writerow({'Location ID': avg_list[i][0],
                             '2Mass ID': avg_list[i][1],
                             '# of Visits': avg_list[i][2],
                             'Average Equivalent Width': avg_list[i][3],
                             'Average Standard Deviation': avg_list[i][4]})
    savefile2.close() #Maybe check this
    
    #OUTPUT
    #f = open(allsavefile, 'w')
    #try:
    #    writer = csv.writer(f,delimiter='\t',lineterminator='\n\n')
    #    for i in range(len(all_list)):
    #        writer.writerow(all_list[i][0],all_list[i][1],all_list[i][2],all_list[i][3],all_list[i][4])
    #finally:
    #    f.close()
    #
    #g = open(avgsavefile, 'w')
    #try:
    #    writer = csv.writer(g,delimiter='\t',lineterminator='\n\n')
    #    for i in range(len(avg_list)):
    #        writer.writerow(avg_list[i][0],avg_list[i][1],avg_list[i][2],avg_list[i][3],avg_list[i][4])
    #finally:
    #   g.close()